In [2]:
import numpy as np
import time

In [3]:
size = 8
ai_player = "O"
human_player = "X"
current_player = human_player
board = np.full((size, size), "-")


def draw_board():
    print("  " + " ".join(str(i) for i in range(size)))
    for i in range(size):
        print(str(i) + " " + " ".join(board[i]))


def get_all_lines():
    lines = []
    lines.extend(board.tolist())  # rows
    lines.extend(board.T.tolist())  # columns
    # diagonals
    for d in range(-size + 1, size):
        lines.append(np.diag(board, k=d).tolist())
        lines.append(np.diag(np.fliplr(board), k=d).tolist())

    lines = [line for line in lines if len(line) >= 4]
    return lines


def check_winner(player):
    lines = get_all_lines()
    win_condition = np.array([player] * 4)

    for line in lines:
        if len(line) >= 4:
            for i in range(len(line) - 3):
                if np.array_equal(line[i : i + 4], win_condition):
                    return True
    return False


def is_goal():
    return check_winner("X") or check_winner("O") or np.all(board != "-")


def hash_board():
    return hash(board.tostring)


def get_all_moves():
    return [(i, j) for i in range(size) for j in range(size) if board[i][j] == "-"]


def reset_board():
    global board
    board = np.full((size, size), "-")

In [4]:
def evaluate():
    score = 0
    lines = get_all_lines()

    for line in lines:

        score += evaluate_line(line, ai_player)
        score += evaluate_line(line, human_player)

    return score


def evaluate_line(line, player):
    score = 0
    result_score = float("-inf") if player == ai_player else float("inf")
    opponent = ai_player if player == human_player else human_player
    for i in range(len(line) - 3):
        window = line[i: i + 4]
        ai_count = window.count(player)
        opponent_count = window.count(opponent)
        empty_count = window.count("-")

        # Điểm cho AI
        if ai_count == 4:
            score += 100000  # Chiến thắng chắc chắn
        elif ai_count == 3 and empty_count == 1:
            score += 5000  # Mở rộng khi có cơ hội chiến thắng
        elif ai_count == 2 and empty_count == 2:
            score += 500  # Tăng điểm để khuyến khích mở rộng thêm
        elif ai_count == 1 and empty_count == 3:
            score += 100  # Khuyến khích phát triển khi có nhiều ô trống

        # Điểm phạt cho đối thủ
        if opponent_count == 3 and empty_count == 1:
            score -= 75000  # Ưu tiên chặn đối thủ với mức điểm phạt nặng nề
        elif opponent_count == 2 and empty_count == 2:
            score -= 2000  # Nặng nề hơn để chặn các nước tiến
        elif opponent_count == 1 and empty_count == 3:
            score -= 200  # Phòng ngừa khi đối thủ bắt đầu phát triển

        if player == ai_player:
            result_score = max(result_score, score)
        else:
            result_score = min(result_score, score)
        
    return result_score


def order_moves(maximum_player):
    moves = get_all_moves()
    scores = []

    # Đánh giá mỗi nước đi
    for move in moves:
        board[move] = ai_player
        score = evaluate()
        scores.append((score, move))
        board[move] = "-"


    scores.sort(reverse=maximum_player)
    print(scores)
    ordered_moves = [move for _, move in scores]
    return ordered_moves

In [5]:
def alpha_beta_pruning(board, depth, alpha, beta, maximum_player):
    if depth == 0 or is_goal():
        return evaluate(), None

    moves = order_moves(maximum_player)
    best_move = None

    for move in moves:
        board[move] = ai_player if maximum_player else human_player
        eval, _ = alpha_beta_pruning(board, depth - 1, alpha, beta, not maximum_player)

        if maximum_player:
            if eval > alpha:
                alpha = eval
                best_move = move
        else:
            if eval < beta:
                beta = eval
                best_move = move

        board[move] = "-"

        if alpha >= beta:
            break

    if maximum_player:
        return alpha, best_move
    else:
        return beta, best_move


def alpha_beta_search(board, limit_depth=3):
    best_move = None
    max_eval = float("-inf")
    alpha = float("-inf")
    beta = float("inf")

    for depth in range(1, limit_depth + 1):
        start = time.time()
        eval, move = alpha_beta_pruning(board, depth, alpha, beta, True)
        if eval > max_eval:
            max_eval = eval
            best_move = move

        end = time.time() - start
        print(
            f"Depth: {depth}, Time: {end:.2f}, Beset eval: {max_eval}, Best move: {best_move}"
        )
    return best_move

In [6]:
board[0][0] = "X"

lines = get_all_lines()

for line in lines:
    for i in range(len(line) - 4):
        sub_line = "".join(line[i: i + 4])
        print(sub_line)

X---
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
X---
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
X---
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
